# Project 3 - Ad-hoc Techniques
Code up the 8-Puzzle using a Genetic Approach and one other non-traditional approaches. Compare against a random approach.

So to start this program, we are going to use our eight puzzle mix up program again as follows

In [1]:
# Moves mapped to 0 = up, 1 = right, 2= down, and 3 = left 
import random
import numpy as np
from utils import memoize

def apply_action(board, action):
    """
    Applies an action to a board
    :param board:  the board which the action is being applied to
    :param action: a integer from 0-3 which corresponds to actions: 0: up, 1:right, 2:down, 3:left
    :return: the board with the new acction applied to it
    """
    deltas = np.array([[-1, 0, 1, 0], [0, 1, 0, -1]])  # changes based on what direction the tile is moved
    posx, posy = np.where(np.isin(board, [0]))  # grabs the position of the blank tile
    (x, y) = (posx[0], posy[0])
    (new_x, new_y) = (x + deltas[0,action], y + deltas[1, action])
    if new_x < 0 or new_y < 0:  # necessary because negative numbers wrap arround
        new_x = 3
    try:
        el = board[new_x,new_y]
        board[x,y] = el
        board[new_x,new_y] = 0
    except IndexError:
        # print('index error')
        pass
    return board


def mess_up(board,moves):
    for iter in range(0,moves):
        board = apply_action(board, random.randint(0, 3)) # generates a random move based on the comment at the top
    pass

In [2]:
def sol_to_string(solution):
    """
    the function takes a solution and returns the names of the moves
    """
    move_dict ={0: 'up', 1:'right', 2:'down', 3:'left'}
    sequence = []
    for x in solution:
        sequence.append(move_dict[x])
    print(sequence)

Let's make a board to play around with. This generates a board with 50 random moves applied to it. For the explaination, we will use this as a global.

In [3]:
board = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
mess_up(board,50)
print(board)

[[1 5 2]
 [8 7 3]
 [4 0 6]]


Next, we want to code up a genetic algorithm solution to the problem. Let's look at how the AIMA looks at genetic algorithims, so we will import the search.py file from the AIMA repository.

In [4]:
from search import *

Let's define our target goal. We did this intially in a function called goal test, which returned true or false based on if our array was equal to our goal. We are going to set target to the goal array.

In [5]:
Target = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

Next we need to define our gene pool. For this problem our gene pool is made of the moves which can be conducted such as left, right, up, down.

In [6]:
gene_pool = [0, 1, 2, 3] #0: up, 1:right, 2:down, 3:left

We then define the max population. I started with 100 for the max population and a 7% mutation rate. I then changed the mutation rate to 1% since there was no need for the mutation rate to be that high.

In [7]:
max_population = 100

In [8]:
mutation_rate = 0.01 # 1%

Next we need to define the fitness. We can use one of the heuristics from earlier as a template. For the fitness I will use a Manhattan distance and the correctness of the top row and left column. These are used in the fitness function. The Manhattan distance is the sum of the total distance horizontally and vertically to the board goal state. So we make a function to calculate the Manhattan distance and a function which calculates the correctness of the top row and the left column. The manhattan distance I can copy over from an earlier project. The fitness value will then be calculated by the following

$Fitness = 1 - Manhattan(0.01) - TopLeft(0.2)$

In [9]:
def manhattan(board):
    """
    grabs the total manhattan distance of the board
    """
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    td = 0
    for item in [1, 2, 3, 4, 5, 6, 7, 8]:
        gx, gy = np.where(np.isin(goal, item))
        cx, cy = np.where(np.isin(board, item))
        td += abs(gx-cx)+abs(gy-cy)
    return td[0]

In [155]:
def topleft(board):
    """
    a 2,1,0 based on the correctness of the top row and column of the board.
    """
    top = 1
    left = 1
    
    if board[0,0] == 1 and board[0,1] == 2 and board[0,2] == 3:
        top = 0
    
    if board[0,0] == 1 and board[1,0] == 4 and board[2,0] == 7:
        left = 0
        
    return top + left

In [11]:
print(manhattan(board), topleft(board))

9 2


In [12]:
def fitness_fn(sample):
    # initialize fitness to 0
    global board
    temp = np.copy(board)
    for x in sample:
        apply_action(temp, x)
    
    return 1 - manhattan(temp)*0.01 - topleft(temp)*0.2

Next, we are going to create our initial population based on the max population and the other factors we chose 31 as our length because 31 steps is the hardest known 8 puzzle.[link](http://w01fe.com/blog/2009/01/the-hardest-eight-puzzle-instances-take-31-moves-to-solve/)

In [13]:
population = init_population(max_population, gene_pool, 31)

Next, we will select 2 parents based on the fitness fuction we defined earlier. 

In [14]:
parents = select(2, population, fitness_fn)

Then we wil use the recombine function in search.py to create a child.

In [15]:
child = recombine(*parents)

Next, we run the mutate function on the child.

In [16]:
child = mutate(child, gene_pool, mutation_rate)

We need to do the above steps on every individual in population to generate an entirely new one.

In [17]:
population = [mutate(recombine(*select(2, population, fitness_fn)), gene_pool, mutation_rate) for i in range(len(population))]

Then grab the best fitting sequence and print it to see it, as well as the fitness function for the best solution

In [18]:
current_best = max(population, key=fitness_fn)
print(current_best)
print(fitness_fn(current_best))
sol_to_string(current_best)

[2, 2, 0, 3, 0, 0, 3, 2, 2, 1, 2, 0, 0, 0, 0, 1, 3, 3, 1, 2, 0, 1, 0, 0, 3, 1, 2, 1, 0, 2, 3]
0.98
['down', 'down', 'up', 'left', 'up', 'up', 'left', 'down', 'down', 'right', 'down', 'up', 'up', 'up', 'up', 'right', 'left', 'left', 'right', 'down', 'up', 'right', 'up', 'up', 'left', 'right', 'down', 'right', 'up', 'down', 'left']


I set the fitness threshold to 9. as well as the max generations to 500. 

In [19]:
fit_thresh = 1.0
mgen = 500

We then run the genetic algorith function which does the previous steps till the fitness threshold is hit or we reach 500 generations. With the current fitness model there could be a problem where a sequence solves the problem but then messes up the puzzle again but this should, for the most part, find a solution

In [20]:
def genetic_algorithm_stepwise(population, fitness_fn, gene_pool=[0, 1], f_thres=None, ngen=1200, pmut=0.1):
    for generation in range(ngen):
        population = [mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut) for i in range(len(population))]
        # stores the individual genome with the highest fitness in the current population
        current_best = (max(population, key=fitness_fn))
        print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')
        
        # compare the fitness of the current best individual to f_thres
        fittest_individual = fitness_threshold(fitness_fn, f_thres, population)
        
        # if fitness is greater than or equal to f_thres, we terminate the algorithm
        if fittest_individual:
            return fittest_individual, generation
    return max(population, key=fitness_fn) , generation

In [21]:
best_fit, generation = genetic_algorithm_stepwise(population, fitness_fn, gene_pool, fit_thresh, mgen, mutation_rate)

In [22]:
def goal_test(board):
    """
    Tests the board to see if it matches the goal state
    :param board: the board which we want to tesy
    :return: true if the board is the same as the goal and fallse if not
    """
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    return np.array_equal(board, goal)

Then we pass the sequence through several functions to shorten to the shortest solution for that sequence.

In [23]:
def shorten_solution(solution):
    short = []
    t_board = np.copy(board) 
    for each in solution:
        short.append(each)
        apply_action(t_board, each)
        if goal_test(t_board):
            return short
        
    return short


short_fit = shorten_solution(best_fit)
print(short_fit)

[0, 3, 1, 3, 2, 1, 3, 3, 0, 2, 1, 0, 3, 2, 2, 0, 1, 0, 1, 3, 1, 2, 0, 0, 0, 2, 2]


I then removed all the moves which did not change the board.

In [24]:
def remove_non(solution):
    short = []
    t_board = np.copy(board)
    c_board = np.copy(board)
    for m in solution:
        apply_action(t_board, m)
        if not np.array_equal(t_board ,c_board):
            short.append(m)
            c_board = np.copy(t_board)

    return short

non_short = remove_non(short_fit)
print(non_short)

[0, 3, 1, 3, 2, 1, 3, 0, 2, 1, 0, 3, 2, 0, 1, 0, 1, 3, 1, 2, 0, 2, 2]


Next I checked the shortened solution for any redundent moves such as 'left' followed by 'right'

In [157]:
def remove_redund(solution):
    short =[]
    key_move = 'none'
    move_map = {0: 2, 1:3, 2:0, 3:1, 'none':'none'}
    for m in solution:
        if m != move_map[key_move]:
            short.append(m)
        else:
            short.pop()
        key_move = m
    return short
    
r_short = remove_redund(non_short)
print(r_short)

[2, 1, 0, 3, 2]


Then im going to make the moves in English so it is easier to follow.

In [158]:
sol_to_string(r_short)

['down', 'right', 'up', 'left', 'down']


I then do a check to make sure the solution actually works.

In [27]:
new_board = np.copy(board)
for x in short_fit:
    apply_action(new_board, x)
    

print(goal_test(new_board))
print(new_board)

True
[[1 2 3]
 [4 5 6]
 [7 8 0]]


# Simulated Annealing
I'm going to program the 8 puzzle through simulated annealing. The first step is to define the temperature function. I'm going to use the exp_schedule function from search.py.

In [97]:
def n_out_of_order(board):
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    return np.count_nonzero(np.subtract(board,goal))

def bad_times_m(board):
    """Multiplies the number of out of order tiles by the manhattan distance"""
    return manhattan(board)*n_out_of_order(board)

The next function is needed to make a list of neighbors to randomly take a move from.

In [29]:
def possiblemoves(board):
    """Returns a list of the possible moves"""
    moves = []
    new_board = np.copy(board)
    c_board = np.copy(board)  # a board to compare to
    for x in range(4):
        new_board = apply_action(new_board, x)
        if not np.array_equal(new_board, c_board):
            moves.append(x)
        new_board = np.copy(c_board)
    return moves

The annealing algorithm uses a cooling function defined as an argument to create a solution. The energy is defined by the Manhattan distance multiplied by the number of tiles out of place. The function returns the board and a tuple of the sequence of moves, which can be reduced later by the functions from earlier. The annealing process selects a move with a lower heuristic. If the move does not have a lower heurstic it can be selected by a probability defined by $a = e^{\frac{c_{new} - c_{old}}{T}}$. This works with the 8 puzzle because sometimes a heuristic will not be able to be reduced with the correct move due to a local minimum. 

In [160]:
import sys
import math
import utils

def eight_anneal(puzzle, schedule=exp_schedule()):
    sequence = []
    c_board = np.copy(board)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return  c_board, tuple(sequence) 
        neighbors = possiblemoves(c_board)
        if not neighbors:
            return c_board, tuple(sequence)
        next_choice = random.choice(neighbors)
        next_board = apply_action(c_board, next_choice)
        sequence.append(next_choice)
        if bad_times_m(next_board)== 0:
            return c_board, tuple(sequence)
        elif bad_times_m(next_board) < bad_times_m(c_board):
            c_board = np.copy(next_board)
        elif utils.probability(math.exp((bad_times_m(next_board) - bad_times_m(c_board))/T)):
            c_board = np.copy(next_board)
        

This next code messes up the board and does one annealing session

In [161]:
board = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
mess_up(board,10)
test, seq = eight_anneal(board)
print(test, '\n', seq, '\n' ,manhattan(test))

[[5 3 2]
 [8 0 1]
 [4 7 6]] 
 (0, 3, 0, 3, 2, 2, 1, 1, 3, 1, 3, 1, 0, 3, 2, 1, 3, 3, 0, 0, 2, 2, 0, 1, 2, 1, 3, 0, 0, 2, 2, 3, 0, 1, 0, 2, 1, 0, 3, 1, 3, 1, 2, 2, 3, 1, 0, 2, 0, 3, 3, 2, 1, 3, 1, 1, 0, 2, 3, 0, 2, 0, 2, 1, 3, 0, 0, 2, 0, 1, 2, 3, 3, 0, 1, 2, 1, 3, 3, 0, 2, 1, 2, 1, 0, 2, 3, 3, 1, 1, 3, 3, 0, 1, 0, 3, 1, 3, 2, 1) 
 12


The following runs the annealing process 7500 times and writes the solutions to a dictionary. For this I do not mind writing over the dictionary entries since I'm only looking for the best/solution sequence of moves.

In [162]:
solutions = {}
for i in range(7500):
    f_board, moves = (eight_anneal(board))
    solutions[bad_times_m(f_board)] = moves

Next we will print the final board, the sequence, and the final heuristic 

In [163]:
print(min(solutions))
print(solutions[min(solutions)])
sol_to_string(solutions[min(solutions)])
best_board = np.copy(board)
for x in solutions[min(solutions)]:
    apply_action(best_board, x)
print(board)

0
(3, 1, 0, 3, 3, 2, 0, 0, 1, 3, 2, 1, 1, 3, 1, 0, 3, 3, 2, 1, 2, 3, 0, 2, 1, 0, 3, 2, 1, 3, 0, 2, 1, 3, 1, 3, 0, 0, 1, 1, 2, 3, 0, 2, 0, 3, 1, 2, 1, 0, 2, 0, 3, 3, 2, 0, 1, 3, 2, 1, 0, 2, 3, 0, 1, 2, 0, 1, 2, 3, 2, 1)
['left', 'right', 'up', 'left', 'left', 'down', 'up', 'up', 'right', 'left', 'down', 'right', 'right', 'left', 'right', 'up', 'left', 'left', 'down', 'right', 'down', 'left', 'up', 'down', 'right', 'up', 'left', 'down', 'right', 'left', 'up', 'down', 'right', 'left', 'right', 'left', 'up', 'up', 'right', 'right', 'down', 'left', 'up', 'down', 'up', 'left', 'right', 'down', 'right', 'up', 'down', 'up', 'left', 'left', 'down', 'up', 'right', 'left', 'down', 'right', 'up', 'down', 'left', 'up', 'right', 'down', 'up', 'right', 'down', 'left', 'down', 'right']
[[1 2 3]
 [4 6 8]
 [7 5 0]]


# Conclusion
For this project I looked at 2 non-traditional problem solving methods; genetic algorithms and simulated annealing. Compared to the random search and the random search with heuristics, I think that each of these methods has its merriots. The genetic algorithm makes a lot of sense for a problem such as the eight puzzle. This is because you select paths which at the end have the best heuristic and then mate them to try and form a better solution. The annealing process is similar to the random with heuristic search but is better since it can move backwards to find a solution.